# Practice Assignment: Understanding Distributions Through Sampling<br>

** *This assignment is optional, and I encourage you to share your solutions with me and your peers in the discussion forums!* **  
<br>

To complete this assignment, create a code cell that:
* Creates a number of subplots using the `pyplot subplots` or `matplotlib gridspec` functionality.
* Creates an animation, pulling between 100 and 1000 samples from each of the random variables (`x1`, `x2`, `x3`, `x4`) for each plot and plotting this as we did in the lecture on animation.
* **Bonus:** Go above and beyond and "wow" your classmates (and me!) by looking into matplotlib widgets and adding a widget which allows for parameterization of the distributions behind the sampling animations.


Tips:
* Before you start, think about the different ways you can create this visualization to be as interesting and effective as possible.
* Take a look at the histograms below to get an idea of what the random variables look like, as well as their positioning with respect to one another. This is just a guide, so be creative in how you lay things out!
* Try to keep the length of your animation reasonable (roughly between 10 and 30 seconds).

In [2]:
%%capture  
%matplotlib widget

# %%capture prevents the output of a cell in jupyter notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from functools import partial
from IPython.display import display
from ipywidgets import interactive, IntSlider, FloatSlider, FloatRangeSlider, Layout


fig, axs = plt.subplots(2, 2, figsize=[8,6])

fig.suptitle('Sampling Four Different Distributions', y=0.96, fontsize=14)

for ax in axs.flat:
    ax.set_xlabel('$x$', fontsize=12)
    ax.set_ylabel('$p(x)$', fontsize=12)
    

fig.tight_layout()



sample_size = IntSlider(value=300, min=100, max=1000, step=20, 
                        description='Sample Size:', style={'description_width': '150px'}, 
                        layout=Layout(width='500px', height='75px', align_items='initial')) 

norm_mean   = FloatSlider(value=0., min=-10., max=10., step=0.5, 
                          description='Normal Dist Mean:', style={'description_width': '150px'}, 
                          layout=Layout(width='500px', height='25px', align_items='initial'))

norm_std    = FloatSlider(value=1., min=0.25, max=5., step=0.25, 
                          description='Normal Dist STD:', style={'description_width': '150px'}, 
                          layout=Layout(width='500px', height='50px', align_items='initial'))

gamma_shape = FloatSlider(value=2., min=0.5, max=10., step=0.5, 
                          description='Gamma Dist Shape:', style={'description_width': '150px'}, 
                          layout=Layout(width='500px', height='25px', align_items='initial'))

gamma_scale = FloatSlider(value=1., min=0.5, max=3., step=0.5, 
                          description='Gamma Dist Scale:', style={'description_width': '150px'}, 
                          layout=Layout(width='500px', height='50px', align_items='initial'))

exp_scale   = FloatSlider(value=1., min=0.25, max=4., step=0.25, 
                          description='Expon Dist Scale:', style={'description_width': '150px'}, 
                          layout=Layout(width='500px', height='50px', align_items='initial'))

uni_range   = FloatRangeSlider(value=[0., 1.], min=-5., max=5., step=0.5, 
                               description='Uniform Dist Range:', style={'description_width': '150px'}, 
                               layout=Layout(width='500px', height='75px', align_items='initial'))

In [3]:
def sampling_dists_ani(frame, samples, sample_size):
    for i, ax in enumerate(axs.flat):
        for rect in ax.patches:
            rect.remove()
            
        ax.hist(samples[i][:frame+1], bins=int(sample_size/20), density=True, color='xkcd:sea blue')
        
        if i == 0:
            ax.texts[-1].remove()
            ax.annotate('N = {}/{}'.format(frame+1, sample_size), [0.86, 0.94], xycoords='figure fraction')
        
    return


def sample_dists(sample_size, norm_mean, norm_std, gamma_shape, gamma_scale, exp_scale, uni_range):
    
    norm_samp = np.random.normal(loc=norm_mean, scale=norm_std, size=sample_size)
    gamm_samp = np.random.gamma(shape=gamma_shape, scale=gamma_scale, size=sample_size)
    expo_samp = np.random.exponential(scale=exp_scale, size=sample_size)
    unif_samp = np.random.uniform(low=uni_range[0], high=uni_range[1], size=sample_size)
    
    dist_samples = [norm_samp, gamm_samp, expo_samp, unif_samp]
    
    axs[0,0].set_xlim([norm_mean-4.*norm_std, norm_mean+4.*norm_std])
    axs[0,1].set_xlim([0., round(max(gamm_samp)*1.2)])
    axs[1,0].set_xlim([0., round(max(expo_samp)*1.2)])
    axs[1,1].set_xlim([uni_range[0]-0.25, uni_range[1]+0.25])
    
    axs[0,0].set_ylim([0., round(max(np.histogram(norm_samp, bins=int(sample_size/20), density=True)[0])*1.3, 1)])
    axs[0,1].set_ylim([0., round(max(np.histogram(gamm_samp, bins=int(sample_size/20), density=True)[0])*1.3, 1)])
    axs[1,0].set_ylim([0., round(max(np.histogram(expo_samp, bins=int(sample_size/20), density=True)[0])*1.3, 1)])
    axs[1,1].set_ylim([0., round(max(np.histogram(unif_samp, bins=int(sample_size/20), density=True)[0])*1.3, 1)])
    
    axs[0,0].annotate('Normal\n\n$\mu$={}\n$\sigma$={}'.format(norm_mean, norm_std), [0.8, 0.68], xycoords='axes fraction', fontsize=9)
    axs[0,0].annotate('N = /{}'.format(sample_size), [0.87, 0.96], xycoords='figure fraction')
    axs[0,1].annotate('Gamma\n\n$k$={}\n'.format(gamma_shape)+r'$\theta$={}'.format(gamma_scale), [0.8, 0.68], xycoords='axes fraction', fontsize=9)
    axs[1,0].annotate('Exponential\n\n'+r'$\beta$={}'.format(exp_scale), [0.71, 0.75], xycoords='axes fraction', fontsize=9)
    axs[1,1].annotate('Uniform\n\nMin={}\nMax={}'.format(uni_range[0], uni_range[1]), [0.78, 0.68], xycoords='axes fraction', fontsize=9)
    axs[1,1].annotate('(c) Tim Gburek', [0.87, 0.03], xycoords='figure fraction', fontsize=8)
    
    ani = animation.FuncAnimation(fig, partial(sampling_dists_ani, samples=dist_samples, sample_size=sample_size), 
                                  frames=sample_size, repeat=False, interval=200)
    
    writer = animation.PillowWriter(fps=24)
    ani.save('sampling_four_distributions.gif', writer=writer)
    
    plt.show()

    return

test = interactive(sample_dists, {'manual': True}, sample_size=sample_size, norm_mean=norm_mean, norm_std=norm_std, 
                                                   gamma_shape=gamma_shape, gamma_scale=gamma_scale, exp_scale=exp_scale, uni_range=uni_range)

display(test)

# widgets_box = widgets.VBox([sample_size, widgets.HBox([norm_mean, norm_std]), widgets.HBox([gamma_shape, gamma_scale]), exp_scale, uni_range, ui.children[-2]])
# widgets.GridBox(ui.children, layout=widgets.Layout(grid_template_columns='550px 550px'))

interactive(children=(IntSlider(value=300, description='Sample Size:', layout=Layout(align_items='initial', he…